In [66]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import logistic
from sklearn.ensemble import RandomForestRegressor

from database_config import cust_list

In [124]:
import seaborn as sns

In [5]:
df1 = pd.read_csv('./data/NA_Report.csv')

In [6]:
df2 = pd.read_csv('./data/products.csv')

In [7]:
df3 = pd.read_csv('./data/Company_wide_NA_Reports.csv')

In [22]:
df2.drop(columns=['customer_id'],inplace=True)

In [15]:
na_report = df1.merge(df2,
          how='inner',
         on='product_id')

In [45]:
foo = na_report.drop(columns=['customer_name','resolution','product_name','image_url','attribute_name','bucket_name','curation_tasks_count','active'])

In [47]:
foo['started_at'] = pd.to_datetime(foo['started_at'])

In [48]:
foo = foo.set_index('curation_task_id')

In [54]:
company_dums = pd.get_dummies(foo['customer_id'])

In [65]:
for col in foo.columns:
    print(col,len(foo[col].unique()))

customer_id 3
started_at 15
product_id 64138
Total_NAs 2
bucket_id 167
external_id 64138


In [75]:
X = foo[['customer_id','bucket_id']]
y = foo['Total_NAs']

In [76]:
ohe = OneHotEncoder()

In [78]:
ohe_X = ohe.fit_transform(X)

C:\Users\groupby\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [82]:
lr = logistic.LogisticRegressionCV()

In [83]:
lr.fit(X_train,y_train)

C:\Users\groupby\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [86]:
lr.score(X_test,y_test)

0.7692906087589791

In [12]:
df3.head()

,customer_id,name,Total_NAs,curation_tasks_count
0,1,back_detailing,0,91
1,1,care_cleaning,0,64
2,1,closure,0,50
3,1,color,0,91
4,1,dress_flow,0,93


In [15]:
le = LabelEncoder()

In [47]:
name_labels = le.fit_transform(df3['name'])

In [48]:
df3.groupby(by=['customer_id','name']).median()

Total_NAs  curation_tasks_count
customer_id name                                                          
1           back_detailing                         0                    91
            care_cleaning                          0                    64
            closure                                0                    50
            color                                  0                    91
            dress_flow                             0                    93
            dress_length                           0                    93
            embellishment                          0                    50
            extended_size_range                    0                    36
            external_resource                      0                    29
            fabric                                 0                    92
            front_detailing                        0                    91
            hemline                                0                    93
            lining                                 0                    79
            neckline                               0                    93
            neckline_old                           0                    90
            newattr1                               0                    91
            occasion                               0                    93
            patterns_prints                        0                    21
            pockets                                0                    92
            silhouette                             0                    92
            sleeve_length                          0                    93
            solids                                 0                    93
            travel_friendly                        0                    82
            waistline                              0                    93
            wrinkle_resistant                      0                    93
2           amperage                               0                   224
            arbor_size                             0                   147
            blade_diameter                         0                   165
            blade_position                         0                    99
            category                               0                   429
...                                              ...                   ...
75          sock_type                              0                  1825
            specialty_padding_type            162811                163952
            specialty_sport_footwear_type     107210                124192
            specialty_sports_bottom_type      162164                162727
            specialty_sports_glove_type       162156                163570
            specialty_sports_tops_type        162156                162156
            strap_count                       162541                162736
            sunglasses_type                    67342                 68492
            support_level                          0                  5931
            swim_bottom_type                   67342                 71727
            swim_cover_up_type                 67342                 67389
            swim_top_type                      67342                 69237
            swimsuit_activity                  67748                 67961
            temperature                        61146                 69071
            tennis_shoe_type                  132533                132533
            top_type                           68094                 88134
            track_field_event                 116873                116873
            training_type                     162156                162156
            underwear_type                     67342                 67818
            upper_material                    108270                126223
            vest_type                          67342                

In [49]:
foo = df3.groupby(by='customer_id').mean()

In [50]:
foo["Average_NA_per_company"] = foo['Total_NAs']/foo['curation_tasks_count']*100

In [51]:
foo.sort_values(by='Average_NA_per_company', ascending=False)

,Total_NAs,curation_tasks_count,Average_NA_per_company
customer_id,,,
75,79808.500000,93296.336364,85.543016
55,21464.651163,26286.837209,81.655511
48,7355.101266,9407.607595,78.182484
65,3667.000000,8179.058824,44.834009
58,9149.121212,20589.181818,44.436546
64,424.617021,1355.191489,31.332622
63,3399.047619,11252.666667,30.206597
34,1786.142132,7258.248731,24.608445
23,913.500000,4005.900000,22.803864


In [52]:
df3['name'] = name_labels

In [53]:
name_labels.shape

(3261,)

In [54]:
df3['name'].shape

(3261,)

In [58]:
ohe = OneHotEncoder()

In [88]:
X = df3['name']
y = df3['Total_NAs']

In [96]:
X_dums = pd.get_dummies(X)

In [92]:
ohe_X.shape

(1, 3261)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X_dums, y, test_size=0.33, random_state=42)

In [98]:
model = RandomForestRegressor(min_samples_leaf=3)

In [99]:
model.fit_transform(X_train,y_train)

C:\Users\groupby\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\groupby\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:127: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [100]:
model.score(X_test,y_test)

-0.0030720980733511816

In [112]:
rank_dict= dict(zip(le.inverse_transform(X_dums.columns),model.feature_importances_))

C:\Users\groupby\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:166: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [123]:
heat = pd.DataFrame(data=rank_dict,index=range(len(rank_dict.keys())))

In [154]:
ranks = heat.sum().sort_values(ascending=True)

In [155]:
NA_ranks = []
for rank in ranks:
    if rank <=0:
       NA_ranks.append(rank) 

In [156]:
ranks[:len(NA_ranks)]

12_volt_dc_output              0.0
placemat_size                  0.0
pizza_pan_size                 0.0
pitcher_capacity               0.0
pillows_throws_type            0.0
pillow_type                    0.0
pillow_size                    0.0
pillow_shape                   0.0
pillar_size                    0.0
piecesin_place_setting         0.0
pieces                         0.0
piece_type                     0.0
piece_size                     0.0
piece                          0.0
pie_type                       0.0
picture_shape                  0.0
picture_capacity               0.0
physical_therapy_aid_type      0.0
phonetype                      0.0
phone_type                     0.0
pet_type                       0.0
pet_parasite_treatment_form    0.0
pet_life_stage                 0.0
pet_items                      0.0
pet_control_accessory_type     0.0
personalized                   0.0
placement                      0.0
plant                          0.0
plant_features      